In [ ]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets import cifar10
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import files
files.upload()

# Kaggle

In [ ]:
# !pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 kaggle.json

In [ ]:
# !kaggle datasets list

In [ ]:
!kaggle datasets download -d jessicali9530/celeba-dataset

In [ ]:
!unzip celeba-dataset.zip

# Generator and Discriminator

In [ ]:
# Define the generator networ.
# This example comsider a CNN. Depending on the nature of the problem, 
# it can be made more complex, and also different models such as VGG can also be used

def Generator(latentShape,imgShape):
    model = Sequential()
    # foundation for 4x4 image
    n_nodes = 256 * 4 * 4
    model.add(Dense(n_nodes, input_shape=latentShape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))
    # upsample to 8x8
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 64X64
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 128X128
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    model.summary()
    
    # opt = RMSprop(learning_rate=0.001)  #Learning rate 
    opt = Adam(0.0002, 0.5)  #Learning rate and momentum.
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [ ]:
def Discriminator(imgShape):

    model = Sequential()
    # normal
    model.add(Conv2D(256, (3,3), padding='same', input_shape=imgShape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    # compile model
    # opt = RMSprop(learning_rate=0.001)
    opt = Adam(lr=0.0002, beta_1=0.5)
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [ ]:
# Combine the generator and discriminator models into one pipeline
def GAN(g, d):
    # Set the discriminator parameters to false
    d.trainable = False
    
    # Create a GAN model = generator + discriminator
    model = Sequential()
    model.add(g)
    model.add(d)   # disciminator takes output of generator as input
    
    # Set compile parameters
    opt = Adam(0.0002, 0.5)  #Learning rate and momentum.
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

# Testing the data

In [ ]:
# (cifar10X, _), (_, _) = cifar10.load_data()

In [ ]:
# cifar10X.shape

In [ ]:
# import tensorflow_datasets as tfds
# ds, ds_info = tfds.load('cifar10', split='train', with_info=True)
# fig = tfds.show_examples(ds, ds_info)
# fig.show()

# Loading Celebrity Face Image Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !pip install scikit-image

In [ ]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize

# path = '/Users/amankesarwani/Downloads/archive/img_align_celeba/img_align_celeba/'
# path = '/content/drive/My Drive/img_align_celeba/'
path = '/content/img_align_celeba/img_align_celeba/'
image_data_arr = []
import random
SEED = 448

imgShape = (128,128,3)

myList = os.listdir(path)
random.seed(SEED)
random.shuffle(myList)
len(myList)

In [ ]:
for i in range(40000):#len(myList):   
#     print(img)
    if(i%1000==0):
        print(i)
    img = myList[i]
    img_array=imread(os.path.join(path,img))
#     print(img_array.shape)
    img_resized=resize(img_array,imgShape)   
    image_data_arr.append(img_resized)

In [ ]:
image_data_arr = np.array(image_data_arr)
# scale the original values [0,255] to [-1,1]
image_data_arr = (image_data_arr.astype('float32') - 0.5) / 0.5

In [ ]:
# img_array=imread(os.path.join(path,img))
# print(img_array.shape)

In [ ]:
# np.array(image_data_arr).shape

In [ ]:
# np.max(np.array(image_data_arr).flatten())

In [ ]:
# np.min(np.array(image_data_arr).flatten())

# Setup

In [ ]:
# output_folder = '/Users/amankesarwani/Documents/DLXR/Deep Learning Advanced/outputrms/'
output_folder = '/content/drive/My Drive/adam/'

In [ ]:
# load the MNIST dataset and change its shape to 28x28x1
def get_real_samples():
    # load MNIST dataset without its class labels
    # (cifar10X, _), (_, _) = cifar10.load_data()
    
    cifar10X = np.array(image_data_arr)
    # scale the original values [0,255] to [-1,1]
    X = (cifar10X.astype('float32') - 0.5) / 0.5
    
    return X

In [ ]:
# While training the discriminator, only a random batch are considered
def get_random_real_samples(dataset, n):
    # choose random n samples
    print(dataset.shape)
    X = dataset[np.random.randint(0, dataset.shape[0], n)]
    
    # Generate Class labels of real samples as 1
    y = np.ones((n, 1))
    return X, y

In [ ]:
# Generate n number of noise samples  for the generator
def generate_noise_samples(noise_dim, n):
    ld=np.prod(noise_dim) # convert noise_dim shape (28,28,1) to 28x28x1
    
    # generate n number of random noise samples of dim ld
    x_input = np.random.normal(0,1, (n, ld))
    # reshape into a batch of inputs for the network
    #x_input = x_input.reshape(n_samples, ld)
    return x_input

In [ ]:
# Generate FAKE samles using Generator from the noise samples
def generate_fake_samples(g, noise_dim, n):

    x_input = generate_noise_samples(noise_dim, n)
    
    # Generate Fake samples and its class label as 0
    X = g.predict(x_input)
    y = np.zeros((n, 1))
    
    return X, y

In [ ]:
import matplotlib.pyplot as plt

def plot_rate(r,f,g,title,i):
 
    plt.figure()
    plt.plot(r,color='green',label='Real',linestyle='--')
    plt.plot(f,color='red',label='Fake',linestyle='--')
    data = np.array([r,f])
    plt.plot(np.average(data, axis=0),color='blue',label='Discriminator')
    plt.plot(g,color='gray',label='Generator')


    plt.xlabel("Epochs")
    plt.ylabel(title)
    plt.legend()

    if(i%100==0):
        filename = output_folder+title+'plot_%d.png' % (i)
        plt.savefig(filename)
        plt.show()
        plt.close()
    else:
        plt.show()
  

In [ ]:
# plt.figure()
# plt.plot([1,2,3],color='green',label='Real')
# plt.plot([3,2,1],color='red',label='Fake')
# # data = np.array([r,f])
# # plt.plot(np.average(data, axis=0),color='blue',label='Discriminator',linestyle='--')
# # plt.plot(g,color='k',label='Generator')


# plt.xlabel("Epochs")
# plt.ylabel('Test')
# plt.legend()
# # plt.show()
# filename = '/content/drive/My Drive/output/'+'Test'+'plot_%d.png' % (1)
# plt.savefig(filename)
# plt.show()
# plt.close()

In [ ]:
# ----------------
# For each epoch, randomely select a batch of real and fake images and train the discriminator and GAN
# ---------------
from IPython.display import clear_output
def train(g, d, gan, dataset, noise_dim, epochs, batch):

    k = int(dataset.shape[0]/batch)
    rl,ra = [],[]
    fl,fa = [],[]
    gl,ga = [],[]
    for i in range(epochs):
        if(i%2==0):
            #print('Waiting to clear')
            clear_output(wait=True)
        for j in range(k):
            # Train discriminator with real and random samples
            #print('Discriminator Training')
            X_real, y_real = get_random_real_samples(dataset, batch)
            r_error, r_acc = d.train_on_batch(X_real, y_real)
            X_fake, y_fake = generate_fake_samples(g, noise_dim, batch)
            f_error, f_acc = d.train_on_batch(X_fake, y_fake)
            
            
            # Train GAN
            #print('GAN Training')
            X_gan = generate_noise_samples(noise_dim, batch)
            y_gan = np.ones((batch, 1))
            g_error, g_acc = gan.train_on_batch(X_gan, y_gan)
            
        # Batch training loss
        rl.append(r_error)
        fl.append(f_error)
        gl.append(g_error)
        ra.append(r_acc)
        fa.append(f_acc)
        ga.append(g_acc)
        
        print('Epoch %d -> Discriminator loss %f GAN loss %f' % (i, (r_error+f_error)/2, g_error))
        print('RE ',r_error,' FE ',f_error,' RA ',r_acc,' FA ',f_acc)
        print('Epoch %d -> Discriminator acc %f GAN acc %f' % (i, (r_acc+f_acc)/2, g_acc))
            
        # Save the generated images every 100 epoch    
        if((i%10==0)):
            save_plot(X_fake,g,i)
        if((i%10==0)):
            save_20plot(X_fake,g,i)
            filename = output_folder+'results_%d.txt' % (i)
            f = open(filename,'w')
            print('Epoch %d -> Discriminator loss %f GAN loss %f' % (i, (r_error+f_error)/2, g_error), file=f)
            print('RE ',r_error,' FE ',f_error,' RA ',r_acc,' FA ',f_acc, file=f)
            print('Epoch %d -> Discriminator acc %f GAN acc %f' % (i, (r_acc+f_acc)/2, g_acc), file=f)
            f.close()
        
        # if((i==3)|(i==5000)|(i==10000)):
        #   plot_rate(rl,fl,gl,'Loss '+str(i))
        #   plot_rate(ra,fa,ga,'Accuracy'+str(i))

        plot_rate(rl,fl,gl,'Loss',i)
        plot_rate(ra,fa,ga,'Accuracy',i)

In [ ]:
# create and save a plot of generated images
def save_plot(fake,g,epoch):
    
    # scale from [-1,1] to [0,1]
    fake = (fake + 1) / 2.0
    # plot images
    
    plt.imshow(fake[0])
    filename = output_folder+'face_%d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()    

In [ ]:
def save_20plot(fake,g,epoch):
    
    # scale from [-1,1] to [0,1]
    fake = (fake + 1) / 2.0
    # plot images
    
    for i in range(20):
        plt.imshow(fake[i])
        filename = output_folder+str(i)+'face_%d.png' % (epoch+1)
        plt.savefig(filename)
        plt.close()

    r,c = 5,5
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(fake[cnt, :,:,0])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(output_folder+'facegrid_%d.png' % epoch)
    plt.close()

# Run

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
# size of the noise input
noiseShape = (1000,)

# create the discriminator
d = Discriminator(imgShape)
# create the generator
g = Generator(noiseShape, imgShape)
# create the gan
gan = GAN(g, d)
# load image data
# dataset = get_real_samples()

In [ ]:
# train model
train(g, d, gan, image_data_arr, noiseShape, 10001, 128)